In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=1,
    google_api_key=os.getenv("GEMINI_API_KEY")
)

In [2]:
from pymongo import MongoClient
CURRENT_USER_ID="67e2862197578c4000ede9f7"

# Kết nối MongoDB
mongo_client = MongoClient(os.getenv("MONGO_URI"))
db = mongo_client["sgtm"]

users_collection = db["users"]
schedules_collection = db["schedules"]

In [5]:
import json
from datetime import datetime

# hàm lấy thông tin collection
def get_schema():
    sample_doc = schedules_collection.find_one()
    if sample_doc:
        # Lấy các trường dữ liệu và format ngày nếu có
        fields = []
        for k, v in sample_doc.items():
            if k != "_id":
                # Kiểm tra xem giá trị có phải là một ngày tháng không
                if isinstance(v, str) and "-" in v:  # Nếu là chuỗi ngày tháng (kiểu "yyyy-mm-dd")
                    try:
                        # Chuyển đổi ngày theo định dạng yyyy-mmm-ddd
                        formatted_date = datetime.strptime(v, "%Y-%m-%d")
                        fields.append(f"{k}: {formatted_date}")
                    except ValueError:
                        fields.append(f"{k}: {v}")  # Nếu không phải ngày tháng hợp lệ thì giữ nguyên giá trị
                else:
                    fields.append(f"{k}: {type(v).__name__}")

        return f"Collection: {schedules_collection.name}\nFields: {', '.join(fields)}"
    return "No schema found"
get_schema()

# Hàm để chuyển đổi các giá trị không tuần tự hóa được (ObjectId, datetime)
def convert_mongo_types(value):
    if isinstance(value, datetime):
        return value.strftime("%Y-%m-%d %H:%M:%S")
    return value

# Hàm lấy thông tin người dùng
def get_user_info():
    user_info = users_collection.find_one({"_id": CURRENT_USER_ID})
    if user_info:
        # Xử lý chuyển đổi toàn bộ dict
        clean_info = {k: convert_mongo_types(v) for k, v in user_info.items()}
        # Bỏ những trường nhạy cảm như password
        clean_info.pop("password", None)
        return json.dumps(clean_info, indent=2, ensure_ascii=False)
    return "No user info found"
print(get_user_info())

{
  "_id": "67e2862197578c4000ede9f7",
  "fullName": "do",
  "birthday": "1990-01-01",
  "email": "thangdo@gmail.com",
  "phone": "0978645132",
  "gender": "MALE",
  "hobbies": "hh",
  "roles": [
    "USER",
    "ADMIN"
  ],
  "createdAt": "2025-03-25 10:32:01",
  "_class": "users",
  "occupation": "IT"
}


In [12]:
from langchain_core.prompts import ChatPromptTemplate

# prompt tạo query theo câu hỏi của người dùng
query_prompt = ChatPromptTemplate.from_template("""
Given the MongoDB schema below, generate the filter part of a MongoDB `find()` query in Python dictionary format that answers the user's question.

The `date` field should be in the format 'YYYY-MM-DD'.

Schema:
{schema}

Question:
{question}

Return only the Python dictionary for the query filter, with no additional formatting, explanations, or backticks.
Ensure the 'date' is in the format 'YYYY-MM-DD'.
MongoDB Query Filter:
""")

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#chain prompt tạo query
def build_query_chain():
    return (
        RunnablePassthrough.assign(schema=lambda _: get_schema())
        | query_prompt
        | llm.bind(stop=["\nResult:"])
        | StrOutputParser()
    )

In [14]:
#ví dụ tạo query theo câu hỏi
response = build_query_chain().invoke({"question": "lúc 13:46:00 ngày 3/4 có sự kiện gì không"})
print(response)

{'date': '2025-04-03', 'startTime': '13:46:00'}


In [9]:
from ast import literal_eval

# hàm chạy query
def run_query(query_code: str):
    try:
        query_dict = literal_eval(query_code)
        result = schedules_collection.find(query_dict)
        return list(result)
    except Exception as e:
        return f"Error running query: {str(e)}"
response = run_query("{'date': {'$regex': '^3/', '$options': 'i'}}")
print(response)

[]


In [ ]:
# prompt main
response_prompt = ChatPromptTemplate.from_template("""
Bạn là trợ lý AI chuyên hỗ trợ người dùng lên lịch trình cá nhân và trả lời các câu hỏi liên quan đến dữ liệu của họ.

🎯 Mục tiêu chính:
- Trả lời tự nhiên, thân thiện, đúng trọng tâm, câu văn cần đầy đủ chủ ngữ vị ngữ.
- Khi người dùng yêu cầu **gợi ý lịch trình**, hãy tạo kế hoạch hợp lý trong ngày, tuân thủ các nguyên tắc sau:

🧩 Quy tắc tạo lịch trình:
- Giới hạn trong khung giờ 07:00–23:00.
- Nếu chỉ yêu cầu **một hoạt động**, chỉ trả về đúng **một gợi ý**, không phải cả lịch trình.
- Không cần chào hỏi đầu câu, trả lời thẳng vào lịch trình.
- Gộp thời gian theo khung dài hợp lý, không chia nhỏ quá (VD: 07:30–11:45 | Làm việc tập trung).
- Luôn có ít nhất một khung **tập thể dục** (VD: 17:30–19:30).
- Trả về theo định dạng sau:

🗓 Lịch trình ngày [ngày/tháng/năm]
[Giờ bắt đầu] - [Giờ kết thúc] | [Tên hoạt động] → [Mô tả]
...
Kết thúc bằng câu: “Bạn có thấy lịch trình này phù hợp không?”

📌 Luật trả lời dữ liệu:
- Nếu người dùng hỏi về thông tin cá nhân (tên, ngày sinh, nghề nghiệp, email,...), hãy sử dụng thông tin có sẵn từ dữ liệu cá nhân.
- Nếu người dùng yêu cầu thông tin liên quan đến lịch trình hoặc các dữ liệu từ MongoDB, sử dụng kết quả truy vấn MongoDB để trả lời.

*lưu ý quan trọng*:
⏰ Thời gian hiện tại: {current_time}

Thông tin có sẵn:
User Info: {user_info}
Schema: {schema}
MongoDB Query: {query}
Query Response: {response}

Yêu cầu người dùng:
{question}

✍️ Trả lời:
""")


In [ ]:
from datetime import datetime

# chain chính
def build_full_chain():
    query_chain = build_query_chain()
    return (
        RunnablePassthrough.assign(query=query_chain)
        .assign(
            user_info=lambda _: get_user_info(),  # Trả về dưới dạng string nếu cần
            schema=lambda _: get_schema(),
            response=lambda vars: run_query(vars["query"]),
            current_time=lambda _: datetime.now().strftime("%d/%m/%Y %H:%M")
        )
        | response_prompt
        | llm
    )


In [ ]:
schedules_chain = build_full_chain()

#ví dụ
question2 = "tháng 4 có lịch đi chơi nào không"
print(schedules_chain.invoke({"question": question2}).content)
